In [12]:
import pandas as pd
import numpy as npy
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)
import datetime
import random
import sqlite3
from sqlalchemy import create_engine

conn = sqlite3.connect('pb_assets/pb_lottery_data.db')

cursor = conn.cursor()

In [13]:
pb_draw_file = ("pb_assets/powerball.csv")
pb_weather_file = ("pb_assets/pb_weather.csv")

pb_draw_df = pd.read_csv(pb_draw_file, engine='python', parse_dates= {"draw_date" : ["year","month","day"]})
pb_weather_df = pd.read_csv(pb_weather_file, engine='python')

pb_weather_df['Date'] = pd.to_datetime(pb_weather_df['DATE_TIME']).dt.date
pb_weather_df['Time'] = pd.to_datetime(pb_weather_df['DATE_TIME']).dt.strftime('%H%M%S')
pb_weather_df = pb_weather_df.dropna()
pb_weather_df = pb_weather_df.astype({'DPTemp':'int', 'DBTemp':'int', 'RHumid':'int'})

pd.to_datetime(pb_draw_df['draw_date'], errors='coerce')
pb_draw_df = pb_draw_df[~(pb_draw_df['draw_date'] < '2015-10-04')]
pb_draw_df['month'] = pd.DatetimeIndex(pb_draw_df['draw_date']).month_name()
pb_draw_df['day_name'] = pd.DatetimeIndex(pb_draw_df['draw_date']).day_name()
pb_draw_df['day_num'] = pd.DatetimeIndex(pb_draw_df['draw_date']).dayofweek
pb_draw_df['day_date'] = pd.DatetimeIndex(pb_draw_df['draw_date']).day
pb_draw_df['draw_time'] = '230000'

pb_pb_draw_df = pb_draw_df[['draw_date', 'pb']].copy()

pb_weather_df.reset_index(drop=True, inplace=True)
pb_draw_df.reset_index(drop=True, inplace=True)
pb_pb_draw_df.reset_index(drop=True, inplace=True)

pb_draw_df.to_sql('pb_draw', conn, if_exists='replace', index = False)
pb_weather_df.to_sql('pb_weather', conn, if_exists='replace', index = False)

print(pb_draw_df)
#print(pb_pb_draw_df)
#print(pb_weather_df)

     draw_date       game  num_1  num_2  num_3  num_4  num_5  pb  multi      month   day_name  day_num  day_date draw_time
0   2015-10-07  Powerball     52     40     48     18     30   9    3.0    October  Wednesday        2         7    230000
1   2015-10-10  Powerball     27     68     12     43     29   1    2.0    October   Saturday        5        10    230000
2   2015-10-14  Powerball     20     15     31     40     29   1    2.0    October  Wednesday        2        14    230000
3   2015-10-17  Powerball     57     62     69     49     48  19    3.0    October   Saturday        5        17    230000
4   2015-10-21  Powerball     57     32     30     42     56  11    4.0    October  Wednesday        2        21    230000
..         ...        ...    ...    ...    ...    ...    ...  ..    ...        ...        ...      ...       ...       ...
928 2023-08-26  Powerball     26     20     63     22     28   5    3.0     August   Saturday        5        26    230000
929 2023-08-28  

In [21]:
weather_filter_df = pb_weather_df.query(("DBTemp > 74 & DBTemp < 78")  and ("RHumid > 78 & RHumid < 82"))

print(weather_filter_df)

weather_date_unique = weather_filter_df['Date'].unique()
weather_date_unique_df = pd.DataFrame(weather_date_unique, columns=['date_time'])
weather_date_unique_df['date_time'] = pd.to_datetime(weather_date_unique_df['date_time'])

weather_date_unique_df['time_only'] = pd.to_datetime(weather_date_unique_df['date_time']).dt.time
weather_date_unique_df['draw_date'] = pd.to_datetime(weather_date_unique_df['date_time']).dt.date
weather_date_unique_df['draw_date'] = pd.to_datetime(weather_date_unique_df['draw_date'])
weather_date_unique_df['draw_date'] = weather_date_unique_df['draw_date'].dt.strftime('%Y-%m-%d')

dates_list = weather_date_unique_df['draw_date'].values.tolist()

dates_list


                 DATE_TIME  DPTemp  DBTemp Precip  RHumid        Date    Time
23     2014-03-13T23:53:00      32      38   0.00      79  2014-03-13  235300
24     2014-03-14T00:53:00      31      37   0.00      79  2014-03-14  005300
49     2014-03-15T01:53:00      33      39   0.00      79  2014-03-15  015300
51     2014-03-15T03:53:00      33      39   0.00      79  2014-03-15  035300
55     2014-03-15T07:53:00      40      46   0.00      79  2014-03-15  075300
...                    ...     ...     ...    ...     ...         ...     ...
83899  2023-03-11T03:53:00      52      58   0.00      81  2023-03-11  035300
83903  2023-03-11T07:53:00      48      54   0.00      80  2023-03-11  075300
83919  2023-03-11T23:53:00      50      56   0.00      81  2023-03-11  235300
83920  2023-03-12T00:53:00      51      57   0.00      81  2023-03-12  005300
83928  2023-03-12T08:53:00      63      69   0.00      81  2023-03-12  085300

[4036 rows x 7 columns]


['2014-03-13',
 '2014-03-14',
 '2014-03-15',
 '2014-03-17',
 '2014-03-18',
 '2014-03-19',
 '2014-03-21',
 '2014-03-22',
 '2014-03-23',
 '2014-03-28',
 '2014-03-31',
 '2014-04-01',
 '2014-04-04',
 '2014-04-05',
 '2014-04-06',
 '2014-04-07',
 '2014-04-08',
 '2014-04-09',
 '2014-04-10',
 '2014-04-11',
 '2014-04-12',
 '2014-04-13',
 '2014-04-14',
 '2014-04-15',
 '2014-04-17',
 '2014-04-19',
 '2014-04-20',
 '2014-04-21',
 '2014-04-23',
 '2014-04-24',
 '2014-04-25',
 '2014-04-28',
 '2014-04-29',
 '2014-04-30',
 '2014-05-01',
 '2014-05-02',
 '2014-05-03',
 '2014-05-04',
 '2014-05-05',
 '2014-05-06',
 '2014-05-07',
 '2014-05-08',
 '2014-05-09',
 '2014-05-10',
 '2014-05-11',
 '2014-05-12',
 '2014-05-13',
 '2014-05-14',
 '2014-05-16',
 '2014-05-17',
 '2014-05-19',
 '2014-05-20',
 '2014-05-21',
 '2014-05-22',
 '2014-05-23',
 '2014-05-24',
 '2014-05-25',
 '2014-05-26',
 '2014-05-27',
 '2014-05-28',
 '2014-05-29',
 '2014-05-30',
 '2014-06-01',
 '2014-06-02',
 '2014-06-03',
 '2014-06-04',
 '2014-06-

In [22]:
arr = pb_draw_df.iloc[:, 2:7].values

pb_draw_df['check'] = npy.any(((arr[:, 1:] - arr[:, :-1]) == 1),axis=1).astype(int)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(pb_draw_df)

pb_draw_df.dtypes

     draw_date       game  num_1  num_2  num_3  num_4  num_5  pb  multi      month   day_name  day_num  day_date draw_time  check
0   2015-10-07  Powerball     52     40     48     18     30   9    3.0    October  Wednesday        2         7    230000      0
1   2015-10-10  Powerball     27     68     12     43     29   1    2.0    October   Saturday        5        10    230000      0
2   2015-10-14  Powerball     20     15     31     40     29   1    2.0    October  Wednesday        2        14    230000      0
3   2015-10-17  Powerball     57     62     69     49     48  19    3.0    October   Saturday        5        17    230000      0
4   2015-10-21  Powerball     57     32     30     42     56  11    4.0    October  Wednesday        2        21    230000      0
5   2015-10-24  Powerball     20     31     56     64     60   2    3.0    October   Saturday        5        24    230000      0
6   2015-10-28  Powerball     56     62     54     63      4  10    2.0    October  Wednes

draw_date    datetime64[ns]
game                 object
num_1                 int64
num_2                 int64
num_3                 int64
num_4                 int64
num_5                 int64
pb                    int64
multi               float64
month                object
day_name             object
day_num               int64
day_date              int64
draw_time            object
check                 int64
dtype: object

In [23]:
counts_1 = pb_draw_df['num_1'].value_counts()
counts_2 = pb_draw_df['num_2'].value_counts()
counts_3 = pb_draw_df['num_3'].value_counts()
counts_4 = pb_draw_df['num_4'].value_counts()
counts_5 = pb_draw_df['num_5'].value_counts()
counts_pb = pb_pb_draw_df['pb'].value_counts()

num_counts_df = pd.concat([counts_1, counts_2, counts_3, counts_4, counts_5], axis=1)
pb_counts_df = pd.concat([counts_pb], axis=1)

num_counts_df.loc[:,'Total'] = num_counts_df.sum(numeric_only=True, axis=1)

num_counts_df.sort_values(by= 'Total', ascending=True, inplace=True)
pd.set_option('display.max_rows', num_counts_df.shape[0]+1)

print(num_counts_df)
print(pb_counts_df)



    num_1  num_2  num_3  num_4  num_5  Total
13     13      8      9     10     10     50
34      7      9     12     12     12     52
49     13      8     12      8     13     54
26     15     12      8      9     11     55
29     12      6     14      6     18     56
46     12     12      8     10     15     57
60      8     14     10      6     20     58
35      8     13     10     13     14     58
4       9     16      7     17      9     58
43      8     14     13     12     12     59
5       9     14     10     12     14     59
51     10      8     12     13     16     59
7      18      8     14     14      6     60
25     13     11     14     12     10     60
9      13     15     16      8      9     61
24     15     12     13     10     11     61
48     11      8     14     12     16     61
31     11     15     15     15      6     62
65     13     12     18     10      9     62
1      11     13     10     13     15     62
14     16      9     12     11     15     63
42     16 

In [24]:
pb_draw_df['draw_date'].astype(str)
pb_pb_draw_df['draw_date'].astype(str)
num_dates_match_df = pb_draw_df[pb_draw_df['draw_date'].isin(dates_list)]
pb_dates_match_df = pb_pb_draw_df[pb_draw_df['draw_date'].isin(dates_list)]

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(num_dates_match_df)
    print(pb_dates_match_df)



     draw_date       game  num_1  num_2  num_3  num_4  num_5  pb  multi      month   day_name  day_num  day_date draw_time  check
0   2015-10-07  Powerball     52     40     48     18     30   9    3.0    October  Wednesday        2         7    230000      0
4   2015-10-21  Powerball     57     32     30     42     56  11    4.0    October  Wednesday        2        21    230000      0
7   2015-10-31  Powerball      9     47     20     25     68   7    2.0    October   Saturday        5        31    230000      0
8   2015-11-04  Powerball     12      2     17     20     65  17    4.0   November  Wednesday        2         4    230000      0
10  2015-11-11  Powerball     26      4     32     55     64  18    3.0   November  Wednesday        2        11    230000      0
12  2015-11-18  Powerball     40     17     46     69     41   6    2.0   November  Wednesday        2        18    230000      0
13  2015-11-21  Powerball     37     57     47     50     52  21    3.0   November   Satur

In [25]:
counts_1 = num_dates_match_df['num_1'].value_counts()
counts_2 = num_dates_match_df['num_2'].value_counts()
counts_3 = num_dates_match_df['num_3'].value_counts()
counts_4 = num_dates_match_df['num_4'].value_counts()
counts_5 = num_dates_match_df['num_5'].value_counts()
counts_pb = num_dates_match_df['pb'].value_counts()
counts_pb_df= pd.DataFrame(counts_pb)
sorted_pb = counts_pb_df.sort_values('pb', ascending=False)
counts_df = pd.concat([counts_1, counts_2, counts_3, counts_4, counts_5], axis=1)
counts_df['totals'] = counts_df.sum(axis=1, skipna=True)
sorted_df = counts_df.sort_values('totals', ascending=False)

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(sorted_df)
    print(sorted_pb)

    num_1  num_2  num_3  num_4  num_5  totals
61     13     15      9     11     13      61
32     18     13      8      6     15      60
28      6     17     12      8     10      53
21      9      9     11     11     10      50
36     13      3     12      9     12      49
23      6     10      8     14     10      48
69      8      9     11      7     13      48
14     12      9      7      8     11      47
20     10      8     12      8      9      47
63      9      8      8      9     12      46
59      9      8     12     14      3      46
39     10      6      5     12     13      46
52      6     12     10      9      9      46
6       8      7      7     11     12      45
41      6      7     11     14      7      45
15      9     12      7      4     13      45
40     18      8      6      8      5      45
62     10     11      6      8      9      44
47      8      7     13      7      8      43
12     13      7     13      4      6      43
16     11      8      8     10    

In [26]:
num_draw_list=(sorted_df.index.values)
pb_draw_list=(sorted_pb.index.values)
print(num_draw_list)
print(pb_draw_list)

[61 32 28 21 36 23 69 14 20 63 59 39 52  6 41 15 40 62 47 12 16  8 64 44
  3 37 55 18 53 33 57 10 54 58 50 42 60 30  5 17 22 26 56 65  2 68 19  7
 66 48 27 38 45 24 67 11 31  1  9 29 51 35 46 49 43 34 25  4 13]
[18  4 26 25  9 14 24 21 13  8 10  2 17 19 22 20 11  7 15  6  5 23  3 16
 12  1]


In [27]:
draw_list = list(num_draw_list)
new_list1 = random.sample(draw_list, 5)

draw_list_pb = list(pb_draw_list)
new_list_pb = random.sample(draw_list_pb, 1)

print(new_list1 + new_list_pb)

[62, 36, 30, 55, 24, 18]
